In [1]:
experimentsPath = "/raid/elhamod/Fish/experiments/" #"/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/Fish/" # "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = "Fish_phylogeny_tripletloss_new_archi_full" #"Fish_tripletloss_alpha_lr_experiments"
device = 0
detailed_reporting = False

In [ ]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange
import wandb

import warnings
# warnings.filterwarnings("ignore")

try:
    import wandb
except:
    print('wandb not found')

from myhelpers import config_plots, TrialStatistics
from myhelpers.try_warning import try_running
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()

experimetnsFileName = "experiments.csv"
WANDB_message="wandb not working"


# For logging to server
try_running(lambda : wandb.login(), WANDB_message)

experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        if experiment_params['image_path'] == 'cifar-100-python':
            datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
        else:
            datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            run = try_running(lambda : wandb.init(project='HGNN', group=experimentName+"-"+experimentHash, name=trialHash, config=row_information), WANDB_message) #, reinit=True

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)

#             try_running(lambda : wandb.watch(model, log="all"), WANDB_message)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

            try_running(lambda : run.finish(), WANDB_message)

        bar.update()

        experiment_index = experiment_index + 1



# if __name__ == "__main__":
#     torch.multiprocessing.set_start_method('spawn')
    
#     import argparse

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--cuda', required=True, type=int)
#     parser.add_argument('--experiments', required=True)
#     parser.add_argument('--data', required=True)
#     parser.add_argument('--name', required=True)
#     parser.add_argument('--detailed', required=False, action='store_true')
#     args = parser.parse_args()
#     main(experimentName=args.name, experimentsPath=args.experiments, dataPath=args.data, device=args.cuda, detailed_reporting=args.detailed)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/7 [00:00<?, ?it/s]

using cuda 0
{'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.01, 'numOfTrials': 1, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': True, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'Fish alpha 0.1', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}
Creating datasets...
{'Alosa chrysochloris': 482298, 'Carassius auratus': 1005907, 'Cyprinus carpio': 429083, 'Esox americanus': 496115, 'Gambusia affinis': 617445, 'Lepisosteus osseus': 519445, 'Lepisosteus platostomus': 731608, 'Lepomis auritus': 1002718, 'Lepomis cyanellus': 476361, 'Lepomis gibbosus': 670266, 'Lepomis gulosus': 476359, 'Lepomis humilis': 892772, 'Lepomis macrochirus': 836783, 'Lepomis megalotis': 271249, 'Lepomis microlophus': 271

{'experimentName': 'Fish_phylogeny_tripletloss_new_archi_full', 'modelName': 'models/78fcadef7f9d40a4d9d12789ca7ca4c756b8660495e53b25b334cad4', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': '64166d8fa41454b367214676437e28256afe8cd18d661d637880cca7', 'trialHash': '78fcadef7f9d40a4d9d12789ca7ca4c756b8660495e53b25b334cad4', 'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.01, 'numOfTrials': 1, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': True, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'Fish alpha 0.1', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}


wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade




iteration:   0%|          | 0/500 [00:00<?, ?it/s]

...

{'experimentName': 'Fish_phylogeny_tripletloss_new_archi_full', 'modelName': 'models/db09c1fb1103170195903a5baafd469fc9dc48cfee2633db0d2fa6e9', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': 'adabce7dec9aebc1e0b2bd518e6ce6164a0f0705eb07215cf79b8983', 'trialHash': 'db09c1fb1103170195903a5baafd469fc9dc48cfee2633db0d2fa6e9', 'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.005, 'numOfTrials': 1, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': True, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'Fish lr 0.05', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}


wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade




iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Training started...




iteration:   0%|          | 0/500 [00:37<?, ?it/s, min_val_loss=inf, train=0.00696, val=0.00661, val_loss=3.64]

iteration:   0%|          | 1/500 [00:37<5:16:01, 38.00s/it, min_val_loss=inf, train=0.00696, val=0.00661, val_loss=3.64]

iteration:   0%|          | 1/500 [03:26<5:16:01, 38.00s/it, min_val_loss=inf, train=0.0807, val=0.0934, val_loss=3.62]  

iteration:   0%|          | 2/500 [03:26<10:39:08, 77.00s/it, min_val_loss=inf, train=0.0807, val=0.0934, val_loss=3.62]

iteration:   0%|          | 2/500 [06:18<10:39:08, 77.00s/it, min_val_loss=10.7, train=0.194, val=0.162, val_loss=3.6]  

iteration:   1%|          | 3/500 [06:18<14:35:40, 105.72s/it, min_val_loss=10.7, train=0.194, val=0.162, val_loss=3.6]

iteration:   1%|          | 3/500 [09:12<14:35:40, 105.72s/it, min_val_loss=6.17, train=0.335, val=0.293, val_loss=3.55]

iteration:   1%|          | 4/500 [09:12<17:22:26, 126.10s/it, min_val_loss=6.17, train=0.335, val=0.293, val_loss=3.55]

iteration:   1%|          | 4/